# Defining a station

In this example we will shortly discuss the use of a station with instruments. A station bounds instruments and has functionality to control and store the settings of the devices. We will start from a simulated station (the virtual dot array) that has a special function `initialize` to directly create the station object with simulated instruments:

In [1]:
from qtt.simulation import virtual_dot_array
station = virtual_dot_array.initialize()

initialize: create virtualdot
initialized virtual dot system (2 dots)


The simulated station contain the instuments; `gates`, `keithley1`, `keithley3`, `keithley4`, `ivvi1`, `ivvi2`, `vawg` and a `digitizer`. In general, every Instrument of a station have Parameter items, which are gettable or settable. For example, using the simulated station we can get the keithley and P1 gate amplitude:

In [8]:
gates = station.gates
print(gates)
print(gates.P1.get())

gates.P1.set(1.5)
print(gates.P1.get())

keithley1 = station.keithley1
print(keithley1)
print(keithley1.amplitude.get())

gates: gates (16 gates)
1.5
1.5
<VirtualMeter: keithley1>
2.99501692636


Default workflow after connecting the instruments (the `initialize` function) is to use the station with the measurement functions. The measurements make use of the instruments in the station. To create a station, you need to define you own `initialize` function. Typically the function looks like:

<pre>
def initialize(reinit=False, server_name=None):
    global station, mwindows, output_map
    
    logging.info('LD400: initialize')
    print('\n')
    
    if _initialized and not reinit:
        return station
    
    # Loading IVVI
    logging.info('LD400: load IVVI driver')
    ivvi = IVVI.IVVI(name='ivvi', address='COM5', server_name=server_name, numdacs=16,)
    
    # Loading Lockins
    logging.info('LD400: load lockin driver')
    lockin1 = SR830.SR830(name='lockin1', address='GPIB1::6::INSTR', server_name=server_name)
    lockin1.output_interface('GPIB')

    # Loading digitizer
    logging.info('LD400: load digitizer driver')
    digitizer = M4i.M4i(name='digitizer', server_name=server_name)

    logging.info('all drivers have been loaded')
    
    
    # Create map for gates and outputs
    gate_map=station_gate_map()
    gates = virtual_gates(name='gates', gate_map=gate_map, server_name=server_name, instruments=[ivvi])

    #Creating the experimental station
    station = qcodes.Station(ivvi, lockin1, digitizer, gates)
    
    logging.info('Initialized LD400 station')   
    return station
</pre>    

A view of all the settings of a instrument or station can be shown using the `snapshot` function. The measurements use `snapshot` to collect the settings, which are stored together with the measurement results.

In [31]:
print(keithley1.snapshot())
print(station.snapshot())

{'functions': {}, 'submodules': {}, '__class__': 'qtt.instrument_drivers.virtual_instruments.VirtualMeter', 'parameters': {'IDN': {'value': None, 'ts': None, 'raw_value': None, '__class__': 'qcodes.instrument.parameter.Parameter', 'full_name': 'keithley1_IDN', 'label': 'IDN', 'instrument': 'qtt.instrument_drivers.virtual_instruments.VirtualMeter', 'instrument_name': 'keithley1', 'name': 'IDN', 'vals': '<Anything>', 'post_delay': 0, 'inter_delay': 0, 'unit': ''}, 'amplitude': {'value': 2.9950169263609085, 'ts': '2018-09-06 11:00:41', 'raw_value': 2.9950169263609085, '__class__': 'qcodes.instrument.parameter.Parameter', 'full_name': 'keithley1_amplitude', 'label': 'keithley1 amplitude', 'instrument': 'qtt.instrument_drivers.virtual_instruments.VirtualMeter', 'instrument_name': 'keithley1', 'name': 'amplitude', 'post_delay': 0, 'inter_delay': 0, 'unit': 'a.u.'}, 'readnext': {'value': None, 'ts': None, 'raw_value': None, '__class__': 'qcodes.instrument.parameter.Parameter', 'full_name': 'k

After all experiments are over, all devices need to be disconnected. The station has a `close` function to stop and clean up all the instrument resources:

In [35]:
virtual_dot_array.close()

close gates: gates (16 gates)
close <VirtualMeter: keithley1>
close <VirtualMeter: keithley3>
close <VirtualMeter: keithley4>
close VirtualIVVI: ivvi1
close VirtualIVVI: ivvi2
close <SimulationAWG: vawg>
close <SimulationDigitizer: sdigitizer>
